In [5]:
import os
import time
import pickle
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

COOKIES_FILE = "cookies_fb.pkl"


def manual_login_and_save_cookies():
    """Abre navegador para login manual y guarda cookies."""
    driver = webdriver.Chrome()
    driver.get("https://www.facebook.com/")

    print("👉 Inicia sesión manualmente en el navegador abierto...")
    print("⏳ Tienes 60 segundos para hacerlo")

    # Te da 60 segundos para completar login
    time.sleep(120)

    # Guardar cookies
    pickle.dump(driver.get_cookies(), open(COOKIES_FILE, "wb"))
    driver.quit()
    print("✅ Cookies guardadas en", COOKIES_FILE)


def load_session_from_cookies():
    """Carga cookies en Requests."""
    cookies = pickle.load(open(COOKIES_FILE, "rb"))
    session = requests.Session()
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'])
    return session


def scrape_autos(session):
    """Ejemplo de scraping: lista títulos de autos"""
    url = "https://www.facebook.com/marketplace"
    response = session.get(url)

    if response.status_code != 200:
        print("⚠️ Error al acceder:", response.status_code)
        return

    from bs4 import BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")
    titulos = [item.get_text(strip=True) for item in soup.select("h2.ui-search-item__title")]

    print("\n🚗 Autos encontrados:")
    for t in titulos:
        print("-", t)


if __name__ == "__main__":
    if not os.path.exists(COOKIES_FILE):
        manual_login_and_save_cookies()

    session = load_session_from_cookies()
    scrape_autos(session)


👉 Inicia sesión manualmente en el navegador abierto...
⏳ Tienes 60 segundos para hacerlo
✅ Cookies guardadas en cookies_fb.pkl

🚗 Autos encontrados:


In [8]:
import pickle

cookies = pickle.load(open("cookies.pkl", "rb"))
print(len(cookies), "cookies guardadas")
for c in cookies[:5]:  # mostrar las 5 primeras
    print(c["name"], "=", c["value"])


24 cookies guardadas
_snoopy = eyJmaW5nZXJwcmludCI6IlNNQWhMcHdESTY2NUpjb09tZ3pldDJ2RTI0U3hlYlRlY29lSkEzMkJTQWwvdFQ4eUdQbG1keHc4aVNqNjBuaDBFdE9LK2taUkNESWtCTC9xTGpBc0RYTU9hOFdqNVNyWHowMzNqYVgvbi9ZdDdJUjVWV3RabzZQTXUzTW9QTFFuek16bEZUQTd4SXR4dDJyd0FSWEI3OVlqUmRNd3BienNVRkN0WXEzZlE4NFRXR3M5QVc0PSIsImtleSI6InB1QlRqUFhib2VNMGwwL1A3cDhSdVJqYTRmZkZMQmd4RnM5cytxUWdLQnZwZXhRSXhsdFJ1L1JmanlhTEpOMC8zbnQzT2ZQOUpLbVhpL2pOSXU2WjBxcGQwbXh3S0JvQyJ9
_mldataSessionId = 8250479a-f7b8-4171-8abf-96bcbef3a0eb
c_ui-navigation = 6.6.146
last_query = 
cp = CL-RM_TUxDQ1BST2NhYjU3


In [12]:
import pickle
import time
import csv
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

COOKIES_FILE = "cookies_fb.pkl"
OUTPUT_CSV = "marketplace_autos.csv"
URL = "https://www.facebook.com/marketplace/category/carros/"

# --- Configurar navegador ---
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument("--start-maximized")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.facebook.com")

# --- Cargar cookies ---
with open(COOKIES_FILE, "rb") as f:
    cookies = pickle.load(f)
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get(URL)
time.sleep(5)  # Esperar carga inicial

# --- Scroll infinito y extracción ---
autos = []
prev_count = 0
scroll_retries = 0
MAX_RETRIES = 5
SCROLL_WAIT = 8  # segundos

while True:
    items = driver.find_elements(By.CSS_SELECTOR, "a[role='link'][href*='/marketplace/item/']")
    for item in items:
        try:
            spans = item.find_elements(By.CSS_SELECTOR, "span[dir='auto']")
            
            price = spans[0].text if len(spans) > 0 else ""
            title = spans[1].text if len(spans) > 1 else ""
            location = spans[2].text if len(spans) > 2 else ""
            km = spans[3].text if len(spans) > 3 else ""
            
            # Evitar duplicados
            if not any(a['titulo'] == title and a['precio'] == price for a in autos):
                autos.append({
                    "titulo": title,
                    "precio": price,
                    "ubicacion": location,
                    "kilometraje": km,
                    "categoria": "Autos y Camionetas",
                    "fecha_extraccion": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                })
        except:
            continue

    # Scroll
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_WAIT)

    if len(autos) == prev_count:
        scroll_retries += 1
        if scroll_retries >= MAX_RETRIES:
            break
    else:
        prev_count = len(autos)
        scroll_retries = 0

print(f"🔢 Total autos encontrados: {len(autos)}")

# --- Guardar CSV ---
with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
    fieldnames = ["titulo", "precio", "ubicacion", "kilometraje", "categoria", "fecha_extraccion"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for auto in autos:
        writer.writerow(auto)

driver.quit()


🔢 Total autos encontrados: 838


PermissionError: [Errno 13] Permission denied: 'marketplace_autos.csv'

In [15]:
import pickle
import csv
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

COOKIES_FILE = "cookies_fb.pkl"
OUTPUT_CSV = "marketplace_autos.csv"
URL = "https://www.facebook.com/marketplace/category/carros/"

# --- Configurar navegador ---
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument("--start-maximized")
# Mantener navegador visible
# options.add_argument("--headless")  # comentar si quieres ver el navegador

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.facebook.com")

# --- Cargar cookies ---
with open(COOKIES_FILE, "rb") as f:
    cookies = pickle.load(f)
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get(URL)

# --- Scroll y extracción ---
autos = []
SCROLL_INCREMENT = 2000  # pixeles por scroll

print("🚀 Iniciando extracción de autos...")

while True:
    items = driver.find_elements(By.CSS_SELECTOR, "a[role='link'][href*='/marketplace/item/']")
    new_found = False

    for item in items:
        try:
            spans = item.find_elements(By.CSS_SELECTOR, "span[dir='auto']")
            price, title, location, km = "", "", "", ""

            for span in spans:
                text = span.text.strip()
                if not text:
                    continue
                if "$" in text:
                    price = text
                elif "km" in text.lower():
                    km = text
                elif title == "":
                    title = text
                elif location == "":
                    location = text

            if not any(a['titulo'] == title and a['precio'] == price for a in autos):
                autos.append({
                    "titulo": title,
                    "precio": price,
                    "ubicacion": location,
                    "kilometraje": km,
                    "categoria": "Autos y Camionetas",
                    "fecha_extraccion": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                })
                new_found = True

                if len(autos) % 50 == 0:
                    print(f"⚡ Autos extraídos hasta ahora: {len(autos)}")
        except:
            continue

    # Scroll automático hasta el final de la página
    driver.execute_script(f"window.scrollBy(0, {SCROLL_INCREMENT});")

    # Si no se cargaron nuevos autos
    if not new_found:
        choice = input("❌ No se cargaron nuevos autos. Presiona Enter para continuar o T + Enter para terminar: ").strip().lower()
        if choice == "t":
            print("🛑 Terminando extracción...")
            break
        else:
            print("⏳ Continuando scroll manual...")

# --- Guardar CSV ---
with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
    fieldnames = ["titulo", "precio", "ubicacion", "kilometraje", "categoria", "fecha_extraccion"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for auto in autos:
        writer.writerow(auto)

print(f"✅ Extracción finalizada. Total autos: {len(autos)}")



🚀 Iniciando extracción de autos...
⏳ Continuando scroll manual...
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 50
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 100
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 150
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 200
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 250
⏳ Continuando scroll manual...
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 300
⏳ Continuando scroll manual...
⏳ Continuando scroll manual...
⏳ Continuando scroll manual...
⏳ Continuando scroll manual...
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 350
⚡ Autos extraídos hasta ahora: 400
⏳ Continuando scroll manual...
⏳ Continuando scroll manual...
⏳ Continuando scroll manual...
⏳ Continuando scroll manual...
🛑 Terminando extracción...
✅ Extracción finalizada. Total autos: 438


In [17]:
import pickle
import csv
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

COOKIES_FILE = "cookies_fb.pkl"
OUTPUT_CSV = "marketplace_camionetas.csv"
URL = "https://www.facebook.com/marketplace/category/camionetas/"

# --- Configurar navegador ---
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument("--start-maximized")
# Mantener navegador visible
# options.add_argument("--headless")  # comentar si quieres ver el navegador

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.facebook.com")

# --- Cargar cookies ---
with open(COOKIES_FILE, "rb") as f:
    cookies = pickle.load(f)
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get(URL)

# --- Scroll y extracción ---
autos = []
#SCROLL_INCREMENT = 2000  # pixeles por scroll

print("🚀 Iniciando extracción de autos...")

while True:
    items = driver.find_elements(By.CSS_SELECTOR, "a[role='link'][href*='/marketplace/item/']")
    new_found = False

    for item in items:
        try:
            spans = item.find_elements(By.CSS_SELECTOR, "span[dir='auto']")
            price, title, location, km = "", "", "", ""

            for span in spans:
                text = span.text.strip()
                if not text:
                    continue
                if "$" in text:
                    price = text
                elif "km" in text.lower():
                    km = text
                elif title == "":
                    title = text
                elif location == "":
                    location = text

            if not any(a['titulo'] == title and a['precio'] == price for a in autos):
                autos.append({
                    "titulo": title,
                    "precio": price,
                    "ubicacion": location,
                    "kilometraje": km,
                    "categoria": "Autos y Camionetas",
                    "fecha_extraccion": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                })
                new_found = True

                if len(autos) % 50 == 0:
                    print(f"⚡ Autos extraídos hasta ahora: {len(autos)}")
        except:
            continue

    # Scroll automático hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #driver.execute_script(f"window.scrollBy(0, {SCROLL_INCREMENT});")
    
    # Si no se cargaron nuevos autos
    if not new_found:
        choice = input("❌ No se cargaron nuevos autos. Presiona Enter para continuar o T + Enter para terminar: ").strip().lower()
        if choice == "t":
            print("🛑 Terminando extracción...")
            break
        else:
            print("⏳ Continuando scroll manual...")

# --- Guardar CSV ---
with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
    fieldnames = ["titulo", "precio", "ubicacion", "kilometraje", "categoria", "fecha_extraccion"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for auto in autos:
        writer.writerow(auto)

print(f"✅ Extracción finalizada. Total autos: {len(autos)}")



🚀 Iniciando extracción de autos...
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 50
⚡ Autos extraídos hasta ahora: 100
⚡ Autos extraídos hasta ahora: 150
⚡ Autos extraídos hasta ahora: 200
⚡ Autos extraídos hasta ahora: 250
⚡ Autos extraídos hasta ahora: 300
⚡ Autos extraídos hasta ahora: 350
⚡ Autos extraídos hasta ahora: 400
⚡ Autos extraídos hasta ahora: 450
⚡ Autos extraídos hasta ahora: 500
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 550
⏳ Continuando scroll manual...
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 600
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 650
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 700
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 750
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 800
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 850
⏳ Continuando scroll manual...
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora:

In [19]:
import pickle
import csv
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

COOKIES_FILE = "cookies_fb.pkl"
OUTPUT_CSV = "marketplace_autos.csv"
URL = "https://www.facebook.com/marketplace/category/carros/"

# --- Configurar navegador ---
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument("--start-maximized")
# Mantener navegador visible
# options.add_argument("--headless")  # comentar si quieres ver el navegador

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.facebook.com")

# --- Cargar cookies ---
with open(COOKIES_FILE, "rb") as f:
    cookies = pickle.load(f)
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get(URL)

# --- Scroll y extracción ---
autos = []
#SCROLL_INCREMENT = 2000  # pixeles por scroll

print("🚀 Iniciando extracción de autos...")

while True:
    items = driver.find_elements(By.CSS_SELECTOR, "a[role='link'][href*='/marketplace/item/']")
    new_found = False

    for item in items:
        try:
            spans = item.find_elements(By.CSS_SELECTOR, "span[dir='auto']")
            price, title, location, km = "", "", "", ""

            for span in spans:
                text = span.text.strip()
                if not text:
                    continue
                if "$" in text:
                    price = text
                elif "km" in text.lower():
                    km = text
                elif title == "":
                    title = text
                elif location == "":
                    location = text

            if not any(a['titulo'] == title and a['precio'] == price for a in autos):
                autos.append({
                    "titulo": title,
                    "precio": price,
                    "ubicacion": location,
                    "kilometraje": km,
                    "categoria": "Autos y Camionetas",
                    "fecha_extraccion": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                })
                new_found = True

                if len(autos) % 50 == 0:
                    print(f"⚡ Autos extraídos hasta ahora: {len(autos)}")
        except:
            continue

    # Scroll automático hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #driver.execute_script(f"window.scrollBy(0, {SCROLL_INCREMENT});")
    
    # Si no se cargaron nuevos autos
    if not new_found:
        choice = input("❌ No se cargaron nuevos autos. Presiona Enter para continuar o T + Enter para terminar: ").strip().lower()
        if choice == "t":
            print("🛑 Terminando extracción...")
            break
        else:
            print("⏳ Continuando scroll manual...")

# --- Guardar CSV ---
with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
    fieldnames = ["titulo", "precio", "ubicacion", "kilometraje", "categoria", "fecha_extraccion"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for auto in autos:
        writer.writerow(auto)

print(f"✅ Extracción finalizada. Total autos: {len(autos)}")



PermissionError: [WinError 5] Acceso denegado: 'C:\\Users\\hfuentes.ca\\.wdm\\drivers\\chromedriver\\win64\\140.0.7339.82\\chromedriver-win32\\chromedriver.exe' -> 'C:\\Users\\hfuentes.ca\\.wdm\\drivers\\chromedriver\\win64\\140.0.7339.82\\chromedriver.exe'

In [1]:
import pickle
import csv
import os
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

COOKIES_FILE = "cookies_fb.pkl"
OUTPUT_CSV = "marketplace_autos.csv"
URL = "https://www.facebook.com/marketplace/category/carros/"
BATCH_SIZE = 100  # <-- cada 100 autos guardamos

# --- Configurar navegador ---
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument("--start-maximized")
# Mantener navegador visible
# options.add_argument("--headless")  # comentar si quieres ver el navegador

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.facebook.com")

# --- Cargar cookies ---
with open(COOKIES_FILE, "rb") as f:
    cookies = pickle.load(f)
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get(URL)

# --- Función para guardar CSV en lotes ---
def guardar_csv(rows, append=True):
    if not rows:
        return
    file_exists = os.path.exists(OUTPUT_CSV)
    mode = "a" if append else "w"
    with open(OUTPUT_CSV, mode, newline="", encoding="utf-8") as f:
        fieldnames = ["titulo", "precio", "ubicacion", "kilometraje", "categoria", "fecha_extraccion"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists or not append:
            writer.writeheader()
        writer.writerows(rows)
    print(f"💾 Guardados {len(rows)} registros en {OUTPUT_CSV}")

# --- Scroll y extracción ---
autos = []
batch = []  # autos por lote

print("🚀 Iniciando extracción de autos...")

while True:
    items = driver.find_elements(By.CSS_SELECTOR, "a[role='link'][href*='/marketplace/item/']")
    new_found = False

    for item in items:
        try:
            spans = item.find_elements(By.CSS_SELECTOR, "span[dir='auto']")
            price, title, location, km = "", "", "", ""

            for span in spans:
                text = span.text.strip()
                if not text:
                    continue
                if "$" in text:
                    price = text
                elif "km" in text.lower():
                    km = text
                elif title == "":
                    title = text
                elif location == "":
                    location = text

            if not any(a['titulo'] == title and a['precio'] == price for a in autos):
                auto_data = {
                    "titulo": title,
                    "precio": price,
                    "ubicacion": location,
                    "kilometraje": km,
                    "categoria": "Autos y Camionetas",
                    "fecha_extraccion": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                }
                autos.append(auto_data)
                batch.append(auto_data)
                new_found = True

                if len(autos) % 50 == 0:
                    print(f"⚡ Autos extraídos hasta ahora: {len(autos)}")

                # Guardar cada 100
                if len(batch) >= BATCH_SIZE:
                    guardar_csv(batch)
                    batch = []
        except:
            continue

    # Scroll automático hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Si no se cargaron nuevos autos
    if not new_found:
        choice = input("❌ No se cargaron nuevos autos. Presiona Enter para continuar o T + Enter para terminar: ").strip().lower()
        if choice == "t":
            print("🛑 Terminando extracción...")
            break
        else:
            print("⏳ Continuando scroll manual...")

# --- Guardar lo que quede ---
if batch:
    guardar_csv(batch)

print(f"✅ Extracción finalizada. Total autos: {len(autos)}")


🚀 Iniciando extracción de autos...
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 50
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 100
💾 Guardados 100 registros en marketplace_autos.csv
⚡ Autos extraídos hasta ahora: 150
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 200
💾 Guardados 100 registros en marketplace_autos.csv
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 250
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 300
💾 Guardados 100 registros en marketplace_autos.csv
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 350
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 400
💾 Guardados 100 registros en marketplace_autos.csv
⚡ Autos extraídos hasta ahora: 450
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 500
💾 Guardados 100 registros en marketplace_autos.csv
⚡ Autos extraídos hasta ahora: 550
⚡ Autos extraídos hasta ahora: 600
💾 Guardados 100 registros en marketplace_au

In [2]:
import pickle
import csv
import os
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

COOKIES_FILE = "cookies_fb.pkl"
OUTPUT_CSV = "marketplace_botes.csv"
URL = "https://www.facebook.com/marketplace/category/boats/"
BATCH_SIZE = 100  # <-- cada 100 autos guardamos

# --- Configurar navegador ---
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument("--start-maximized")
# Mantener navegador visible
# options.add_argument("--headless")  # comentar si quieres ver el navegador

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.facebook.com")

# --- Cargar cookies ---
with open(COOKIES_FILE, "rb") as f:
    cookies = pickle.load(f)
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get(URL)

# --- Función para guardar CSV en lotes ---
def guardar_csv(rows, append=True):
    if not rows:
        return
    file_exists = os.path.exists(OUTPUT_CSV)
    mode = "a" if append else "w"
    with open(OUTPUT_CSV, mode, newline="", encoding="utf-8") as f:
        fieldnames = ["titulo", "precio", "ubicacion", "kilometraje", "categoria", "fecha_extraccion"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists or not append:
            writer.writeheader()
        writer.writerows(rows)
    print(f"💾 Guardados {len(rows)} registros en {OUTPUT_CSV}")

# --- Scroll y extracción ---
autos = []
batch = []  # autos por lote

print("🚀 Iniciando extracción de autos...")

while True:
    items = driver.find_elements(By.CSS_SELECTOR, "a[role='link'][href*='/marketplace/item/']")
    new_found = False

    for item in items:
        try:
            spans = item.find_elements(By.CSS_SELECTOR, "span[dir='auto']")
            price, title, location, km = "", "", "", ""

            for span in spans:
                text = span.text.strip()
                if not text:
                    continue
                if "$" in text:
                    price = text
                elif "km" in text.lower():
                    km = text
                elif title == "":
                    title = text
                elif location == "":
                    location = text

            if not any(a['titulo'] == title and a['precio'] == price for a in autos):
                auto_data = {
                    "titulo": title,
                    "precio": price,
                    "ubicacion": location,
                    "kilometraje": km,
                    "categoria": "Embarcaciones",
                    "fecha_extraccion": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                }
                autos.append(auto_data)
                batch.append(auto_data)
                new_found = True

                if len(autos) % 50 == 0:
                    print(f"⚡ Autos extraídos hasta ahora: {len(autos)}")

                # Guardar cada 100
                if len(batch) >= BATCH_SIZE:
                    guardar_csv(batch)
                    batch = []
        except:
            continue

    # Scroll automático hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Si no se cargaron nuevos autos
    if not new_found:
        choice = input("❌ No se cargaron nuevos autos. Presiona Enter para continuar o T + Enter para terminar: ").strip().lower()
        if choice == "t":
            print("🛑 Terminando extracción...")
            break
        else:
            print("⏳ Continuando scroll manual...")

# --- Guardar lo que quede ---
if batch:
    guardar_csv(batch)

print(f"✅ Extracción finalizada. Total autos: {len(autos)}")


🚀 Iniciando extracción de autos...
🛑 Terminando extracción...
💾 Guardados 10 registros en marketplace_botes.csv
✅ Extracción finalizada. Total autos: 10


In [ ]:
import pickle
import csv
import os
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

COOKIES_FILE = "cookies_fb.pkl"
OUTPUT_CSV = "marketplace_camione_1.csv"
URL = "https://www.facebook.com/marketplace/category/search/?query=camiones"
BATCH_SIZE = 100  # <-- cada 100 autos guardamos

# --- Configurar navegador ---
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument("--start-maximized")
# Mantener navegador visible
# options.add_argument("--headless")  # comentar si quieres ver el navegador

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.facebook.com")

# --- Cargar cookies ---
with open(COOKIES_FILE, "rb") as f:
    cookies = pickle.load(f)
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get(URL)

# --- Función para guardar CSV en lotes ---
def guardar_csv(rows, append=True):
    if not rows:
        return
    file_exists = os.path.exists(OUTPUT_CSV)
    mode = "a" if append else "w"
    with open(OUTPUT_CSV, mode, newline="", encoding="utf-8") as f:
        fieldnames = ["titulo", "precio", "ubicacion", "kilometraje", "categoria", "fecha_extraccion"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists or not append:
            writer.writeheader()
        writer.writerows(rows)
    print(f"💾 Guardados {len(rows)} registros en {OUTPUT_CSV}")

# --- Scroll y extracción ---
autos = []
batch = []  # autos por lote

print("🚀 Iniciando extracción de autos...")

while True:
    items = driver.find_elements(By.CSS_SELECTOR, "a[role='link'][href*='/marketplace/item/']")
    new_found = False

    for item in items:
        try:
            spans = item.find_elements(By.CSS_SELECTOR, "span[dir='auto']")
            price, title, location, km = "", "", "", ""

            for span in spans:
                text = span.text.strip()
                if not text:
                    continue
                if "$" in text:
                    price = text
                elif "km" in text.lower():
                    km = text
                elif title == "":
                    title = text
                elif location == "":
                    location = text

            if not any(a['titulo'] == title and a['precio'] == price for a in autos):
                auto_data = {
                    "titulo": title,
                    "precio": price,
                    "ubicacion": location,
                    "kilometraje": km,
                    "categoria": "Camiones y buses",
                    "fecha_extraccion": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                }
                autos.append(auto_data)
                batch.append(auto_data)
                new_found = True

                if len(autos) % 50 == 0:
                    print(f"⚡ Autos extraídos hasta ahora: {len(autos)}")

                # Guardar cada 100
                if len(batch) >= BATCH_SIZE:
                    guardar_csv(batch)
                    batch = []
        except:
            continue

    # Scroll automático hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Si no se cargaron nuevos autos
    if not new_found:
        choice = input("❌ No se cargaron nuevos autos. Presiona Enter para continuar o T + Enter para terminar: ").strip().lower()
        if choice == "t":
            print("🛑 Terminando extracción...")
            break
        else:
            print("⏳ Continuando scroll manual...")

# --- Guardar lo que quede ---
if batch:
    guardar_csv(batch)

print(f"✅ Extracción finalizada. Total autos: {len(autos)}")


🚀 Iniciando extracción de autos...
⏳ Continuando scroll manual...
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 50
⏳ Continuando scroll manual...
⚡ Autos extraídos hasta ahora: 100
💾 Guardados 100 registros en marketplace_camione.csv
⚡ Autos extraídos hasta ahora: 150
⚡ Autos extraídos hasta ahora: 200
💾 Guardados 100 registros en marketplace_camione.csv
⚡ Autos extraídos hasta ahora: 250
⚡ Autos extraídos hasta ahora: 300
💾 Guardados 100 registros en marketplace_camione.csv
⚡ Autos extraídos hasta ahora: 350
⚡ Autos extraídos hasta ahora: 400
💾 Guardados 100 registros en marketplace_camione.csv
⚡ Autos extraídos hasta ahora: 450
⚡ Autos extraídos hasta ahora: 500
💾 Guardados 100 registros en marketplace_camione.csv
⚡ Autos extraídos hasta ahora: 550
⚡ Autos extraídos hasta ahora: 600
💾 Guardados 100 registros en marketplace_camione.csv
⚡ Autos extraídos hasta ahora: 650
⚡ Autos extraídos hasta ahora: 700
💾 Guardados 100 registros en marketplace_camione.csv
⚡ Autos extr